In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
import re
from nltk import ngrams
from collections import defaultdict
import nltk
import collections
import torch

In [14]:
# Read data 
X_train = open('x_train.txt',encoding="utf8").read().split('\n')
Y_train = open('y_train.txt',encoding="utf8").read().split('\n')
labels = pd.read_csv('labels.csv',delimiter = ';')
Y_train =  Y_train[:10000]
X_train =  X_train[:10000]

In [15]:
# Remove unnecessary characters from data
extras = '!"$%&/{}[]()=?\\`´*+~#-_.:,;<>|1234567890°-\'' # Characters to remove from data
rx = '[' + re.escape(''.join(extras)) + ']'
x_train =[] 
for example in X_train:
    x_train.append(re.sub(' +', ' ',re.sub(rx, '', example)))

In [16]:
# Convert language labels to language Name => 'en' -> 'English'
lab_dict = { labels.loc[i]['Label']:labels.loc[i]['English'] for i in range(0,len(labels))}
y_train = [lab_dict[item] if item != 'nan' else 'Min Nan Chinese' for item in Y_train ]

In [7]:
# Collecting examples belonging to same language
# lang_2_ex = { 'English' : [ 'Welcome to ..', 'PHP sucks ...', ...] , 'Spanish' : ['dads....', 'dasdsa...',....] , ....}

lang_2_ex = defaultdict(list)
for i in range(len(x_train)):
    lang_2_ex[y_train[i]].append(x_train[i])

In [8]:
# Creating ngrams for each language 
n=3 # trigram
gram_per_lang = defaultdict(list)
uniq_lang = set(y_train)
for lang in uniq_lang:
    for sent in lang_2_ex[lang]:
        gram_per_lang[lang]+=[sent[i:i+n] for i in range(len(sent)-n+1)]

In [9]:
# Calculating frequency distribution for ngrams per language

freq_per_lang = defaultdict(list)
for lang in uniq_lang:
    freq_per_lang[lang]=dict(zip(list(collections.Counter(gram_per_lang[lang]).keys()),list(collections.Counter(gram_per_lang[lang]).values()))) 

# https://stackoverflow.com/questions/12282232/how-do-i-count-unique-values-inside-a-list
# https://stackoverflow.com/questions/209840/how-do-i-convert-two-lists-into-a-dictionary

In [10]:
# Sorted Frequency Distribution 

sorted_freq_per_lang = defaultdict(list)
for lang in uniq_lang:
    sorted_freq_per_lang[lang] = {word : value for word,value in sorted(freq_per_lang[lang].items(),key=lambda item : item[1],reverse=True)}

In [11]:
# Top 30 trigrams from each language

features = []
for lang, grams_dict in sorted_freq_per_lang.items():
    i = 0
    for gram,count in grams_dict.items():
        if i <=40:
            features.append(gram)
        else:
            break
        i+=1
    
features = list(set(features))

n = len(x_train)
m = len(features)
mat = np.zeros((n,m))
mat.shape



(10000, 4283)

In [17]:
i = 0
for sent in x_train:
    trigrams = [sent[i:i+3] for i in range(len(sent)-3+1)]
    tri_dict = dict(zip(collections.Counter(trigrams).keys(),collections.Counter(trigrams).values()))
    gram_count = []
    for gram in features:
        if gram in tri_dict.keys():
            gram_count.append(tri_dict[gram]+1)
        else:
            gram_count.append(1)
    mat[i] = gram_count
    i+=1

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
encoder = LabelEncoder()
y = encoder.fit_transform(y_train)
X_,x_test,Y_,y_test = train_test_split(mat,y,test_size=0.2,random_state=42)
model = GaussianNB()
model.fit(X_,Y_)
x,y = x_test,y_test
y_pred = model.predict(x)
conf_matrix = confusion_matrix(y_pred=y_pred,y_true=y)
acc = round(accuracy_score(y_pred=y_pred,y_true=y),2) * 100
print(f"Accuracy is {acc}%")

Accuracy is 80.0%
